In [1]:
import gmaps
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as st

# Google developer API key
from Config import gkey

# Access maps with unique API key
gmaps.configure(api_key=gkey)


In [2]:
#create cordinates for NY State map
coordinates = [(42.9915, -76.0718)]


In [3]:
# Customize the size of the figure
figure_layout = {
    'width': '500px',
    'height': '500px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

fig = gmaps.figure(zoom_level = 6.5, center = (42.9915, -76.0718), layout = figure_layout, map_type="HYBRID")
fig

Figure(layout=FigureLayout(border='1px solid black', height='500px', margin='0 auto 0 auto', padding='1px', wi…

In [4]:
AQI_df = pd.read_csv('../Se7en/AQIndex.csv')
AQI_df = AQI_df.dropna()
aqi_group = AQI_df.groupby ("Year")
aqi_2009_df = aqi_group.get_group (2009)
aqi_2009_df.head()

,Unnamed: 0,zip,AQI,Category Number,Category Name,Pollutant,Year
0,0,12032,90,2,Moderate,OZONE,2009
1,1,12108,90,2,Moderate,OZONE,2009
2,2,12134,90,2,Moderate,OZONE,2009
3,3,12139,90,2,Moderate,OZONE,2009
4,4,12164,90,2,Moderate,OZONE,2009


In [5]:
#introducing zip code coordinates
zip_df = pd.read_csv('../Se7en/us-zip-code-latitude-and-longitude.csv')
zip_df = zip_df.rename (columns = {"Zip" : "zip"})

#take care of NaN velue
zip_df = zip_df.dropna()
zip_df.head()

,zip,City,State,Latitude,Longitude,Timezone,Daylight savings time flag
0,10001,New York,NY,40.750742,-73.99653,-5,1
1,10002,New York,NY,40.717040,-73.98700,-5,1
2,10003,New York,NY,40.732509,-73.98935,-5,1
3,10005,New York,NY,40.706019,-74.00858,-5,1
4,10006,New York,NY,40.707904,-74.01342,-5,1


In [10]:
#introducing Vehicle data
vehicle_df = pd.read_csv("vehicle_data.csv.gz", compression="gzip")

#take care of NaN velue
vehicle_df = vehicle_df.dropna()

#creating list of cars
vehicle_year_df = vehicle_df.loc[vehicle_df['model_year'].astype(int) <= "2009"]
vehicle_year_df.head
#creating dataframe for cars age 2009 and older
#veh_age = year - vehicle_year_df['model_year'].astype(int)
#new_vehicle_df = pd.DataFrame(vehicle_year_df)
#new_vehicle_df['vehicle_age'] = veh_age
#vehicle_df.head()

TypeError: Invalid comparison between dtype=int32 and str

In [7]:
# merge the 2 dataframes on zipcode
new_aqi_df = pd.merge(aqi_2009_df, zip_df, on=['zip','zip'], how="left" )


#take care of NaN velue
new_aqi_df = new_aqi_df.dropna()
new_aqi_df.head()

,Unnamed: 0,zip,AQI,Category Number,Category Name,Pollutant,Year,City,State,Latitude,Longitude,Timezone,Daylight savings time flag
0,0,12032,90,2,Moderate,OZONE,2009,Caroga Lake,NY,43.158567,-74.49646,-5.0,1.0
1,1,12108,90,2,Moderate,OZONE,2009,Lake Pleasant,NY,43.525388,-74.40875,-5.0,1.0
2,2,12134,90,2,Moderate,OZONE,2009,Northville,NY,43.218920,-74.14822,-5.0,1.0
3,3,12139,90,2,Moderate,OZONE,2009,Piseco,NY,43.402760,-74.56972,-5.0,1.0
4,4,12164,90,2,Moderate,OZONE,2009,Speculator,NY,43.515611,-74.37431,-5.0,1.0


In [7]:
#Plot Heatmap
location = new_aqi_df[["Latitude", "Longitude"]]
aqi = new_aqi_df["AQI"].astype(float)

#Create layer
heat_layer = gmaps.heatmap_layer(location, weights=aqi, 
                                 dissipating=False, max_intensity=500,                                 
                                 point_radius=0.05)

# Add layer
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='500px', margin='0 auto 0 auto', padding='1px', wi…